# Importing Libraryies

* System Append to set proper path

In [1]:
sys.path.append('../')

* Default

In [2]:
import lasio
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter

* Pandas Option

In [3]:
pd.set_option('max_columns', None)

* Tqdm Progress Bar

In [4]:
%%capture
from tqdm import tqdm_notebook

# Checkpoint Import

In [5]:
df = pd.read_csv('../checkpoints/total_df.csv.gz', compression='gzip')

In [6]:
df.head()

,CALI,NPHI,RHOB,GR,DTC,RDEP,LITHOLOGY_GEOLINK,DEPTH,WELL_NAME
0,19.406000,0.475704,1.789,64.497482,167.582153,1.574993,NaN,493.493134,15_9-12
1,19.406000,0.479429,1.754,62.406261,167.425064,1.569011,NaN,493.645538,15_9-12
2,19.406000,0.474963,1.778,62.629055,167.808395,1.578010,NaN,493.797943,15_9-12
3,19.452999,0.504394,1.642,65.998596,169.244873,1.586024,NaN,493.950348,15_9-12
4,19.452999,0.480163,1.563,64.997223,170.635086,1.603011,NaN,494.102722,15_9-12


# Lithology Code Prediction

In [7]:
litho_data = df[pd.notnull(df['LITHOLOGY_GEOLINK'])].drop(columns=['WELL_NAME']) # not null dataframe (model training)

In [8]:
len(litho_data)

1294715

    * Converting Lithology Data to Integer

In [9]:
litho_data['LITHOLOGY_GEOLINK'] = litho_data['LITHOLOGY_GEOLINK'].astype(int)

    * Reducing Dataset Size

In [10]:
litho_data["Set"] = np.random.choice(["train_red", "rest"], p =[.6, .4], size=(litho_data.shape[0],))

train = litho_data[litho_data.Set == 'train_red']

In [11]:
print(train.shape, litho_data.shape)

(777173, 9) (1294715, 9)


    * Dataset Split

In [12]:
from sklearn.model_selection import train_test_split

X = train.drop(columns=['LITHOLOGY_GEOLINK', 'Set'])

Y = train['LITHOLOGY_GEOLINK']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=42)

In [13]:
print('### Training Dataset: ### ', x_train.shape, y_train.shape)
print('### Validation Dataset: ### ', x_val.shape, y_val.shape)
print('### Test Dataset: ### ', x_test.shape, y_test.shape)

### Training Dataset: ###  (497390, 7) (497390,)
### Validation Dataset: ###  (124348, 7) (124348,)
### Test Dataset: ###  (155435, 7) (155435,)


In [14]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

x_train = std_scaler.fit_transform(x_train)
x_val = std_scaler.fit_transform(x_val)
x_test = std_scaler.fit_transform(x_test)

    * Hyper-Parameter Tunning --> Random Search

        * Parameter Grid Definition

In [16]:
grid = {
    #"n_a": [4, 16, 32, 64],
    #"n_d": [4, 16, 32, 64],
    "gamma": [1, 1.2, 1.5, 2],
    #"n_steps": [3, 5, 7]}
     "momentum": [0.6, 0.9, 0.98],
     "lambda_sparse": [0, 1e-6, 1e-2, 0.1]
}

        * TabNet Call

In [17]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

        * Grid Search (n_experiment)

In [18]:
from sklearn.model_selection import ParameterGrid

test_grid =list(ParameterGrid(grid))

grid_result = pd.DataFrame()

mean_loss = []

mean_train_acc = []

mean_valid_acc = []

max_loss = []

max_train_acc = []

max_valid_acc = []

for param_grid in tqdm_notebook(test_grid, desc='Parameter Grid Search:'):

    print(param_grid)

    tabnet_rand = TabNetClassifier(**param_grid) # generating model

    tabnet_rand.fit(X_train=x_train, y_train=y_train,eval_set=[(x_train, y_train),(x_val, y_val)], eval_name=['train', 'valid'], eval_metric=['balanced_accuracy'], max_epochs=20, num_workers=25, drop_last=False, batch_size=32768, virtual_batch_size=16384, pin_memory=False) # fitting

    if len(grid_result) == 0:
        grid_result = pd.DataFrame.from_records([param_grid]) # storing first configuration

    else:
        grid_result = grid_result.append(param_grid, ignore_index=True) # storing configuration used on this iteration

    mean_loss.append(np.mean(tabnet_rand.history['loss']))

    mean_train_acc.append(np.mean(tabnet_rand.history['train_balanced_accuracy']))

    mean_valid_acc.append(np.mean(tabnet_rand.history['valid_balanced_accuracy']))

    max_loss.append(np.max(tabnet_rand.history['loss']))

    max_train_acc.append(np.max(tabnet_rand.history['train_balanced_accuracy']))

    max_valid_acc.append(np.max(tabnet_rand.history['valid_balanced_accuracy']))

    del tabnet_rand

grid_result['Mean Loss'] = mean_loss

grid_result['Mean Train Acc'] = mean_train_acc

grid_result['Mean Valid Acc'] = mean_train_acc

grid_result['Max Loss'] = max_loss

grid_result['Max Train Acc'] = max_train_acc

grid_result['Max Valid Acc'] = max_valid_acc

{'gamma': 1, 'lambda_sparse': 0, 'momentum': 0.6}
Device used : cuda
Stop training because you reached max_epochs = 20 with best_epoch = 18 and best_valid_balanced_accuracy = 0.27634
Best weights from best epoch are automatically used!
{'gamma': 1, 'lambda_sparse': 0, 'momentum': 0.9}
Device used : cuda
Stop training because you reached max_epochs = 20 with best_epoch = 18 and best_valid_balanced_accuracy = 0.27606
Best weights from best epoch are automatically used!
{'gamma': 1, 'lambda_sparse': 0, 'momentum': 0.98}
Device used : cuda
Stop training because you reached max_epochs = 20 with best_epoch = 18 and best_valid_balanced_accuracy = 0.27607
Best weights from best epoch are automatically used!
{'gamma': 1, 'lambda_sparse': 1e-06, 'momentum': 0.6}
Device used : cuda
Stop training because you reached max_epochs = 20 with best_epoch = 19 and best_valid_balanced_accuracy = 0.30234
Best weights from best epoch are automatically used!
{'gamma': 1, 'lambda_sparse': 1e-06, 'momentum': 0.

In [19]:
grid_result.sort_values(by='Mean Valid Acc', ascending=False).head(n=10)

,gamma,lambda_sparse,momentum,Mean Loss,Mean Train Acc,Mean Valid Acc,Max Loss,Max Train Acc,Max Valid Acc
15,1.2,0.000001,0.60,1.578161,0.244331,0.244331,2.837538,0.303355,0.305228
16,1.2,0.000001,0.90,1.578161,0.243982,0.243982,2.837538,0.301986,0.303507
17,1.2,0.000001,0.98,1.578161,0.243903,0.243903,2.837538,0.301656,0.302661
12,1.2,0.000000,0.60,1.584692,0.239481,0.239481,2.833939,0.293467,0.289666
13,1.2,0.000000,0.90,1.584692,0.239256,0.239256,2.833939,0.292083,0.289504
14,1.2,0.000000,0.98,1.584692,0.239189,0.239189,2.833939,0.291401,0.289057
3,1.0,0.000001,0.60,1.604694,0.232773,0.232773,2.821493,0.299990,0.302340
4,1.0,0.000001,0.90,1.604694,0.232577,0.232577,2.821493,0.302146,0.304383
5,1.0,0.000001,0.98,1.604694,0.232554,0.232554,2.821493,0.302587,0.303855
21,1.2,0.100000,0.60,1.671097,0.232407,0.232407,2.905556,0.284185,0.286468


In [20]:
grid_result.to_csv('../models/results/grid_performance_search.csv')